# PROJET MUSHROOM

## This is our main for our AI projet in python, for the milestone.

We start by clearing the json and making it readable in a good format

In [1]:
!pip install Pillow

In [2]:
import pandas as pd

file_path = 'champiURL.json'
df = pd.read_json(file_path)
df = df.explode('images', ignore_index = True)
df = df.drop(columns=['info'])
mapping = {  'c' : 1,  'p' : 0}
df['comestibility'] = df['comestibility'].map(mapping)
df = df.dropna().reset_index()
df['comestibility'] = df['comestibility'].astype(int)
df

In [3]:
from PIL import Image
from io import BytesIO
import numpy as np
import requests
import os

if not os.path.exists('images'):
   # Create the directory if it doesn't exist
  os.makedirs('images')
for i in range(len(df)):
   # Make a GET request to the URL
   url = df['images'][i]
   response = requests.get(url)
   response.raise_for_status()  # Check if the request was successful

  if response.status_code == 200:
     # Convert the image to base64
    image = Image.open(BytesIO(response.content))
   image = image.resize((64, 64))
  image.save(f'images/{i+1}.png')
else:
   # If the request was not successful, print an error message
  print(f'Error: Unable to fetch image from {url}. Status code: {response.status_code}')

   if i%50 == 0:
      print(f'Image {i}/{len(df)}')


<DirectView [0, 1, 2, 3,...]>

###Simple publication.

Here is a simple Python function we are going to run on the Engines.  This function uses `publish_data` to publish a simple Python dictionary when it is run.

In [4]:
def publish_it():
    from IPython.kernel.zmq.datapub import publish_data
    publish_data(dict(a='hi'))


We run the function on the Engines using `apply_async` and save the returned `AsyncResult` object:


In [5]:
ar = dv.apply_async(publish_it)


The published data from each engine is then available under the `.data` attribute of the `AsyncResult` object.


In [6]:
ar.data

[{'a': 'hi'},
 {'a': 'hi'},
 {'a': 'hi'},
 {'a': 'hi'},
 {'a': 'hi'},
 {'a': 'hi'},
 {'a': 'hi'},
 {'a': 'hi'}]


Each time `publish_data` is called, the `.data` attribute is updated with the most recently published data.
## Simulation loop


In many cases, the Engines will be running a simulation loop and we will want to publish data at each time step of the simulation.  To show how this works, we create a mock simulation function that iterates over a loop and publishes a NumPy array and loop variable at each time step. By inserting a call to `time.sleep(1)`, we ensure that new data will be published every second.


In [7]:
def simulation_loop():
    from IPython.kernel.zmq.datapub import publish_data
    import time
    import numpy as np
    for i in range(10):
        publish_data(dict(a=np.random.rand(20), i=i))
        time.sleep(1)


Again, we run the `simulation_loop` function in parallel using `apply_async` and save the returned `AsyncResult` object.


In [8]:
ar = dv.apply_async(simulation_loop)


New data will be published by the Engines every second.  Anytime we access `ar.data`, we will get the most recently published data.


In [9]:
import matplotlib.pyplot as plt

data = ar.data
for i, d in enumerate(data):
    plt.plot(d['a'], label='engine: '+str(i))
plt.title('Data published at time step: ' + str(data[0]['i']))
plt.legend()

In [9]:
IMAGE